# Data Cleaning

## 1. Set up

### 1.1 Import Libraries

In [0]:
from utils import data_exploration as dexplor
from utils import config
import importlib
importlib.reload(config)
importlib.reload(dexplor)


### 1.2 Load Dataset

In [0]:
%run "./00_Data_Loading"

## 2. Initial Exploration 

### 2.1 Shape of the Datasets 

In [0]:
# Games DataFrame
print("Games DF")
print(games_df.count(), len(games_df.columns))

# Games Metadata DataFrame
print("Games Metadata DF")
print(games_metadata_df.count(), len(games_metadata_df.columns))

# Recommendations DataFrame
print("Recommendations DF")
print(recommendations_df.count(), len(recommendations_df.columns))

# Users DataFrame
print("Users DF")
print(users_df.count(), len(users_df.columns))


### 2.2 Preview the Data

In [0]:
# Games DataFrame
print("Games DF")
display(games_df.limit(5)) 

# Games Metadata DataFrame
print("Games Metadata DF")
display(games_metadata_df.limit(5)) 

# Recommendations DataFrame
print("Recommendations DF")
display(recommendations_df.limit(5)) 

# Users DataFrame
print("Users DF")
display(users_df.limit(5)) 

### 2.3 Data Types and Schema 

In [0]:
# Games DataFrame
print("Games DF")
games_df.printSchema()

# Games Metadata DataFrame
print("Games Metadata DF")
games_metadata_df.printSchema()


# Recommendations DataFrame
print("Recommendations DF")
recommendations_df.printSchema()


# Users DataFrame
print("Users DF")
users_df.printSchema()


### 2.4 Duplicate Rows Check

Check for json data frame 

In [0]:


# Games DataFrame
print("Games DF")
dexplor.checkDuplicates(games_df)

# Games Metadata DataFrame
print("Games Metadata DF")
dexplor.checkDuplicates(games_metadata_df)

# Recommendations DataFrame
print("Recommendations DF")
dexplor.checkDuplicates(recommendations_df)

# Users DataFrame
print("Users DF")
dexplor.checkDuplicates(users_df)

Check for json data frame 

In [0]:
# Games DataFrame
# Check if a game list listed more than once
dexplor.check_duplicates_by(games_df, ["app_id", "title"], "games_df")

# Games Metadata DataFrame
# Check if a game list listed more than once
dexplor.check_duplicates_by(games_metadata_df, ["app_id", "description"], "games_metadata_df")

# Recommendations DataFrame
# Check if a user gave more than one review for a game
dexplor.check_duplicates_by(recommendations_df, ["user_id", "app_id"], "recommendations_df")

# Users DataFrame 
# Check if user listed more than once 
dexplor.check_duplicates_by(users_df, ["user_id"], "users_df")


Check for json data frame 

In [0]:

# Games DataFrame
print("Games DF")
dexplor.checkSuspiciousStrings(games_df)

# Games Metadata DataFrame
print("Games Metadata DF")
dexplor.checkSuspiciousStrings(games_metadata_df)

# Recommendations DataFrame
print("Recommendations DF")
dexplor.checkSuspiciousStrings(recommendations_df)

# Users DataFrame
print("Users DF")
dexplor.checkSuspiciousStrings(users_df)

###2.5 Null/Missing Values

Check for null rows 


In [0]:
# Games DataFrame
print("Games DF")
dexplor.checkNullRows(games_df)

# Games Metadata DataFrame
print("Games Metadata DF")
dexplor.checkNullRows(games_metadata_df)

# Recommendations DataFrame
print("Recommendations DF")
dexplor.checkNullRows(recommendations_df)

# Users DataFrame
print("Users DF")
dexplor.checkNullRows(users_df)

### 2.6 Summary Statistics

In [0]:
def getNumericColumns(df):
    numeric_types = ('int', 'double', 'float', 'decimal')
    return [name for name, dtype in df.dtypes if any(dtype.startswith(nt) for nt in numeric_types)]



# Games DataFrame
print("Games DF")
games_df.select(getNumericColumns(games_df)).describe().show()

# Recommendations DataFrame
print("Recommendations DF")
recommendations_df.select(getNumericColumns(recommendations_df)).describe().show()

# Users DataFrame
print("Users DF")
users_df.select(getNumericColumns(users_df)).describe().show()

### 2.7 Unique Values / Cardinality

In [0]:
from pyspark.sql.functions import countDistinct


def getCardinality(df):
    cardinality_df = df.select([
    countDistinct(col(c)).alias(f"{c}_unique") for c in df.columns
])
    cardinality_df.show()
    return cardinality_df



# Games DataFrame
print("Games DF")
getCardinality(games_df)

# Games Metadata DataFrame
print("Games Metadata DF")
getCardinality(games_metadata_df)

# Recommendations DataFrame
print("Recommendations DF")
getCardinality(recommendations_df)

# Users DataFrame
print("Users DF")
getCardinality(users_df)


### 2.8 Date range check 

In [0]:
from pyspark.sql.functions import col, min, max
from pyspark.sql.types import DateType, TimestampType

def getDateRange(df):
    date_cols = [field.name for field in df.schema.fields 
                 if isinstance(field.dataType, (DateType, TimestampType))]

    if not date_cols:
        print("No date or timestamp columns found.")
        return

    date_range_df = df.select([
        min(col(c)).alias(f"{c}_min") for c in date_cols
    ] + [
        max(col(c)).alias(f"{c}_max") for c in date_cols
    ])

    date_range_df.show(truncate=False)


# Games DataFrame
print("Games DF")
getDateRange(games_df)

# Recommendations DataFrame
print("Recommendations DF")
getDateRange(recommendations_df)




### 2.9 Data Range Check

In [0]:
from pyspark.sql.functions import col, min, max
from pyspark.sql.types import NumericType, DateType, TimestampType

def get_column_ranges(df):
    """
    Prints min and max for each numeric, date, or timestamp column in the given PySpark DataFrame.
    """
    # Identify relevant columns
    target_cols = [
        field.name for field in df.schema.fields
        if isinstance(field.dataType, (NumericType, DateType, TimestampType))
    ]

    if not target_cols:
        print("No numeric or date columns found.")
        return

    # Create min and max aggregation expressions
    agg_exprs = []
    for col_name in target_cols:
        agg_exprs.append(min(col(col_name)).alias(f"{col_name}_min"))
        agg_exprs.append(max(col(col_name)).alias(f"{col_name}_max"))

    # Show the result
    df.select(agg_exprs).show(truncate=False)


get_column_ranges(games_df)
get_column_ranges(recommendations_df)


### 2.10 Value Distribution for Categorical/Boolean Fields

In [0]:
def get_categorical_distribution(df, col_name):
    """
    Prints the distribution (value counts) for a categorical column in descending order.
    
    Args:
        df (DataFrame): PySpark DataFrame
        col_name (str): Name of the categorical column
    
    Returns:
        None (prints the distribution)
    """
    df.groupBy(col_name).count() \
      .orderBy("count", ascending=False) \
      .show(truncate=False)




In [0]:
# Games DF
get_categorical_distribution(games_df, "rating")
get_categorical_distribution(games_df, "win")
get_categorical_distribution(games_df, "mac")
get_categorical_distribution(games_df, "linux")
get_categorical_distribution(games_df, "steam_deck")

# Recommendations DF 

get_categorical_distribution(recommendations_df, "is_recommended")
